## Emulation Queries

Requires the following to be performed on the DB:

```
CREATE EXTENSION tablefunc;
```

The following demonstrates how to generate a dynamic parameter table for a given model where each `run_id` has a column for its various parameters.

In [1]:
from sqlalchemy.sql import text

import sys
sys.path.append("../db")

from database import init_db, db_session, engine
from models import Metadata, Output, Parameters

import pandas as pd

postgres


Generate that returns a string which dynamically builds the columns resulting from the crosstab:

In [2]:
col_query = """SELECT CONCAT('run_id VARCHAR, ', params, ' VARCHAR') 
               FROM (SELECT string_agg(DISTINCT parameter_name::text, ' VARCHAR, ') 
               AS params FROM parameters) AS ps"""
print(col_query)
conn = engine.connect()
rs = conn.execute(col_query)
dynamic_cols = rs.first()[0]
print("\nDynamic column list to inject into query:")
print(dynamic_cols)

SELECT CONCAT('run_id VARCHAR, ', params, ' VARCHAR') 
               FROM (SELECT string_agg(DISTINCT parameter_name::text, ' VARCHAR, ') 
               AS params FROM parameters) AS ps

Dynamic column list to inject into query:
run_id VARCHAR, crop VARCHAR, irrigation VARCHAR, nitrogen VARCHAR, stat VARCHAR


Select the model of interest:

In [3]:
model = 'yield_anomalies_lpjml'

Generate dynamic cross tab query:

In [4]:
ct = f"""SELECT
  *
FROM
  crosstab (
     $$SELECT DISTINCT run_id, parameter_name, parameter_value FROM parameters 
     WHERE model = '{model}'
     GROUP BY run_id, parameter_name, parameter_value
     ORDER BY 1$$,
    $$SELECT DISTINCT parameter_name FROM parameters ORDER BY 1$$
  ) AS (
    {dynamic_cols}
) ORDER BY run_id
"""

Read into pandas DataFrame:

In [5]:
df = pd.read_sql_query(ct, conn)

In [6]:
df

,run_id,crop,irrigation,nitrogen,stat
0,03aa57b2ae8a1a9ba087d7d1266f90db8c8dc62f842e05...,wheat,NO,LIM,"pctl,95"
1,1098aef84395584d99fed198da4c167590c003db05622b...,wheat,LIM,LIM_p25,"pctl,5"
2,25afdf9b7d25dcb14f7ec3d082a1a326f651219160f03d...,wheat,LIM,LIM_p50,mean
3,2a5717269b3758392bca2eb209b5efe78e34f5e49a3ced...,wheat,NO,LIM_p25,"pctl,5"
4,3a211b54c4c8a67c7c4a1094a03a12de4b7760e77b0cd2...,wheat,NO,LIM_p25,"pctl,95"
5,3b6c5b4f513fe73f94c5cd550749eda5cb3a53205ec906...,wheat,LIM,UNLIM,std
6,3db90226196258dd9c06b78be410b8461948b4d2a6f3c6...,wheat,LIM,UNLIM,"pctl,95"
7,45aff92c2a8190acd5ebb2cd5662fb28a5db83385a75a2...,wheat,LIM,LIM_p50,std
8,477a8f5db3a9084418d1d833c771e0c505b44fb4f23344...,wheat,LIM,LIM,"pctl,5"
9,5c433914f10ef5763e67eed8fe1ea11c92f3d3883dd9e8...,wheat,NO,LIM,"pctl,5"
